In [2]:
##Helper function to sort mnist targets => 0123456789
def sort_by_target(mnist):
    ## Use of List Comprehension ([10*x for x in range (5)] == > returns list directly without creating a result variable)
    ## sorted() will sort list in ascending order (sorts tuples by first element, then second element, etc)
    ## enumerate() will return a tuple (index,value)
    ## [:,1] will return the second column (which is the index column)
    ## Note: We have two lists (train, and test) in order to have the last 10,000 elements be specified for testing
    reorder_train = np.array(sorted([(target, i) for i, target in enumerate(mnist.target[:60000])]))[:, 1]
    reorder_test = np.array(sorted([(target, i) for i, target in enumerate(mnist.target[60000:])]))[:, 1]
    mnist.data[:60000] = mnist.data[reorder_train]
    mnist.target[:60000] = mnist.target[reorder_train]
    mnist.data[60000:] = mnist.data[reorder_test + 60000] ## np.array([1,2,3]) + 1 ==> [2,3]
    mnist.target[60000:] = mnist.target[reorder_test + 60000] ## np.array([1,2,3]) + 1 ==> [2,3]

In [3]:
import numpy as np
mnist.target = mnist.target.astype(np.int8) # fetch_openml() returns targets as strings
sort_by_target(mnist) # fetch_openml() returns an unsorted dataset

In [4]:
X,y = mnist.data,mnist.target
X_train,X_test,y_train,y_test = X[:60000],X[60000:],y[:60000],y[60000:]

In [5]:
from scipy.ndimage import shift

def shift_image (image, dy ,dx):
    image = image.reshape(28,28)
    image = shift(image, [dy,dx]).reshape([-1])
    return image

In [7]:
for dy,dx in ((0,1),(0,-1),(1,0),(-1,0)):
    for image,label in zip(X_train,y_train):
        X_train_augmented.append(shift_image(image,dy,dx))
        y_train_augmented.append(label)

In [6]:
X_train_augmented = [image for image in X_train]
y_train_augmented = [image for image in y_train]

In [9]:
shuffle_index = np.random.permutation(len(X_train_augmented))
X_train_augmented, y_train_augmented = X_train_augmented[shuffle_index], y_train_augmented[shuffle_index]

In [8]:
X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

In [10]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_jobs = 10, weights='uniform', n_neighbors=5) ## got best hyperparameters from exercise 1

In [11]:
knn_model.fit(X_train_augmented,y_train_augmented)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=10, n_neighbors=5, p=2,
                     weights='uniform')

In [12]:
y_predictions = knn_model.predict(X_test)

In [13]:
from sklearn.metrics import f1_score
f1_score(y_test,y_predictions,average='macro') ## 0.4% increaseh

0.9753556814013731

In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True)